In [4]:
import requests
from bs4 import BeautifulSoup

def get_first_google_search_link(query):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    google_search_url = 'https://www.google.com/search'
    params = {'q': query}
    
    response = requests.get(google_search_url, headers=headers, params=params)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        search_results = soup.find_all('a', href=True)
        for result in search_results:
            href = result['href']
            if "https://www.linkedin.com/in" in href:
                return href
    return None

# Example usage
search_string = "Srikaran Boya MIT linkedin"
first_link = get_first_google_search_link(search_string)

# Extract the URL part
start = first_link.find("url=") + 4
end = first_link.find("&", start)
extracted_url = first_link[start:end]

print(extracted_url)

https://www.linkedin.com/in/srikaran-reddy
